![](https://i.ibb.co/SvMwJk3/gtr2.jpg)

In [ ]:
!gdown --id 18lHOUi_yBw3ITZ0QK2v4Zz8S3hsXmDtd
!gdown --id 1rLM9m-wNklIOr2dpaOUgW6SrzP3qkUGV

Downloading...
From: https://drive.google.com/uc?id=18lHOUi_yBw3ITZ0QK2v4Zz8S3hsXmDtd
To: /content/lastfm_user_scrobbles.csv.zip
100% 471k/471k [00:00<00:00, 31.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rLM9m-wNklIOr2dpaOUgW6SrzP3qkUGV
To: /content/lastfm_artist_list.csv
100% 330k/330k [00:00<00:00, 48.3MB/s]


In [ ]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
import re
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set_style("darkgrid")
%matplotlib inline

In [ ]:
df_scrobbles = pd.read_csv('/content/lastfm_user_scrobbles.csv.zip')
df_artist = pd.read_csv('/content/lastfm_artist_list.csv')

In [ ]:
def get_info(data):
    display(data.head().style.set_caption('Dataset info'))
    display(data.describe().T)
    print('----')
    print('Other info')
    print('----')
    print()
    display(data.info())
    print('----')
    print()

In [ ]:
get_info(df_scrobbles)

,user_id,artist_id,scrobbles
0,1,4562,13883
1,1,10191,11690
2,1,494,11351
3,1,6673,10300
4,1,8402,8983


,count,mean,std,min,25%,50%,75%,max
user_id,92792.0,945.175845,546.755311,1.0,471.0,945.0,1417.0,1892.0
artist_id,92792.0,8440.215687,4879.112200,1.0,4037.0,8460.0,12435.0,17493.0
scrobbles,92792.0,745.521521,3752.146187,1.0,107.0,260.0,614.0,352698.0


----
Other info
----

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92792 entries, 0 to 92791
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   user_id    92792 non-null  int64
 1   artist_id  92792 non-null  int64
 2   scrobbles  92792 non-null  int64
dtypes: int64(3)
memory usage: 2.1 MB


None

----



In [ ]:
get_info(df_artist)

,artist_id,artist_name
0,1,__Max__
1,2,_Algol_
2,3,-123 Min.
3,4,-Oz-
4,5,-T De Sangre


,count,mean,std,min,25%,50%,75%,max
artist_id,17493.0,8747.0,5049.938465,1.0,4374.0,8747.0,13120.0,17493.0


----
Other info
----

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17493 entries, 0 to 17492
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_id    17493 non-null  int64 
 1   artist_name  17493 non-null  object
dtypes: int64(1), object(1)
memory usage: 273.5+ KB


None

----



In [ ]:
df_scrobbles.groupby('user_id').count().mean().to_frame('counts, %')

,"counts, %"
artist_id,49.044397
scrobbles,49.044397


In [ ]:
df_artist.index = df_artist['artist_id']
artist_dict = df_artist['artist_name'].to_dict()

In [ ]:
#UI

rows, r_pos = np.unique(df_scrobbles.values[:, 0], return_inverse=True)
cols, c_pos = np.unique(df_scrobbles.values[:, 1], return_inverse=True)

In [ ]:
df_sparse = sparse.csr_matrix((df_scrobbles.values[:, 2], (r_pos, c_pos)))

In [ ]:
df_sparse

<1892x17493 sparse matrix of type '<class 'numpy.longlong'>'
	with 92723 stored elements in Compressed Sparse Row format>

In [ ]:
# similar

pui = normalize(df_sparse, norm='l2', axis=1)
sim = pui.T * pui

In [ ]:
# with cos-dist
sim.todense()

matrix([[0.00120405, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.00258841, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.00383689, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.0228355 , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.00827076,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.0010429 ]])

Guns N' Roses = id 6240

In [ ]:
[artist_dict[i+1] for i in sim[6239].toarray().argsort()[0][-20:]]

['Black Sabbath',
 'Skid Row',
 'Van Halen',
 'Nirvana',
 'Kiss',
 'The Doors',
 'System Of A Down',
 'Bon Jovi',
 'Megadeth',
 'Red Hot Chili Peppers',
 'Pink Floyd',
 'Mötley Crüe',
 'Iron Maiden',
 'Metallica',
 'Aerosmith',
 'Queen',
 'Led Zeppelin',
 'Ac/Dc',
 'The Beatles',
 "Guns N' Roses"]

In [ ]:
df_sparse_transposed = df_sparse.transpose(copy=True)
piu = normalize(df_sparse_transposed, norm='l2', axis=1)

In [ ]:
fit = pui * piu * pui

In [ ]:
fit.shape

(1892, 17493)

In [ ]:
data = [artist_dict[i+1] for i in np.nonzero(df_sparse[520])[1].tolist()]

In [ ]:
preds = [artist_dict[i+1] for i in fit[520].toarray().argsort()[0][-70:].tolist()]

In [ ]:
set(preds) - set(data)

{'30 Seconds To Mars',
 'Britney Spears',
 'Christina Aguilera',
 'Coldplay',
 'David Bowie',
 'Duran Duran',
 'Evanescence',
 'Flyleaf',
 'Him',
 'Iron Maiden',
 'Joy Division',
 'Kings Of Leon',
 'Lady Gaga',
 'Linkin Park',
 'Madonna',
 'Marilyn Manson',
 'Metallica',
 'Muse',
 'Nightwish',
 'Nine Inch Nails',
 'Oasis',
 'Opeth',
 'Pink Floyd',
 'Porcupine Tree',
 'Radiohead',
 'Rammstein',
 'System Of A Down',
 'The Beatles',
 'The Killers',
 'The Smashing Pumpkins',
 'The Smiths',
 'U2'}